In [1]:
!del /Q utils\__pycache__

In [2]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
from utils.datasets import GhlKasperskyDataset, TepHarvardDataset, TepKasperskyDataset, SwatItrustDataset
from utils.custom_plots import plot_stacked
from utils.metrics import time_span_metrics

In [4]:
from sklearn.metrics import mean_squared_error

# Data

In [5]:
ds = TepHarvardDataset()
ds.shake_not_stir()

In [6]:
train, _ , _ = next(ds.train_generator())

In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
scaler = StandardScaler()
for data, _, _ in ds.train_generator():
    scaler.partial_fit(data)

In [9]:
SEED = 7145

# LSTM

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Conv1D, ConvLSTM1D
from tensorflow.keras.backend import clear_session
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import set_random_seed

In [11]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [12]:
n_features = train.shape[1]
valid_size = 0.2
res_lstm = pd.DataFrame(dtype='float')

In [13]:
checkpoint = ModelCheckpoint('best_tep1_lstm.hdf5',
                             monitor='val_mse', 
                             verbose=0, 
                             mode='min',
                             save_best_only = True,
                            )

earlystop = EarlyStopping(monitor='val_mse',
                          patience=2,
                          min_delta = 0.001,
                          restore_best_weights=True,
                          verbose=0,
                         )

callbacks_list = [checkpoint, earlystop]

In [14]:
for n_steps in tqdm([2, 4, 8, 16, 32]):
    for rnn_dim in tqdm([32, 64, 128, 256]):
        res_lstm.loc[n_steps, rnn_dim] = 0.0
        clear_session()
        set_random_seed(SEED)
        model = Sequential()
        model.add(LSTM(rnn_dim, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
        model.add(LSTM(rnn_dim, activation='relu'))
        model.add(Dense(n_features))
        model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])
        for train, _, _ in ds.train_generator():
            X, y = split_sequences(scaler.transform(train), n_steps)

            train_n = round(X.shape[0] * (1 - valid_size))
            X_train = X[:train_n]
            y_train= y[:train_n]
            X_valid = X[train_n:]
            y_valid = y[train_n:]

            model.fit(X_train, y_train,
                      epochs=100,
                      validation_data=(X_valid, y_valid),
                      callbacks=callbacks_list,
                      verbose=0,
                     )

            y_ = model.predict(X_valid)
            res_lstm.loc[n_steps, rnn_dim] += mean_squared_error(y_valid, y_)
res_lstm /= len(ds.train_generator())

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [15]:
res_lstm

,32,64,128,256
2,0.506590,0.492223,0.488855,0.484421
4,0.498227,0.480511,0.468784,0.467806
8,0.502821,0.472968,0.464410,0.465121
16,0.508891,0.475423,0.465474,0.461862
32,0.502266,0.467115,0.459850,0.463456


In [16]:
res_lstm.mean(axis=0)

32     0.503759
64     0.477648
128    0.469475
256    0.468533
dtype: float64

In [17]:
res_lstm.mean(axis=1)

2     0.493022
4     0.478832
8     0.476330
16    0.477913
32    0.473172
dtype: float64

# GRU

In [18]:
res_gru = pd.DataFrame(dtype='float')

In [19]:
checkpoint = ModelCheckpoint('best_tep1_gru.hdf5',
                             monitor='val_mse', 
                             verbose=0, 
                             mode='min',
                             save_best_only = True,
                            )

earlystop = EarlyStopping(monitor='val_mse',
                          patience=2,
                          min_delta = 0.001,
                          restore_best_weights=True,
                          verbose=0,
                         )

callbacks_list = [checkpoint, earlystop]

In [20]:
for n_steps in tqdm([2, 4, 8, 16, 32]):
    for rnn_dim in tqdm([32, 64, 128, 256]):
        res_gru.loc[n_steps, rnn_dim] = 0.0
        clear_session()
        set_random_seed(SEED)
        model = Sequential()
        model.add(GRU(rnn_dim, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
        model.add(GRU(rnn_dim, activation='relu'))
        model.add(Dense(n_features))
        model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])
        for train, _, _ in ds.train_generator():
            X, y = split_sequences(scaler.transform(train), n_steps)

            train_n = round(X.shape[0] * (1 - valid_size))
            X_train = X[:train_n]
            y_train= y[:train_n]
            X_valid = X[train_n:]
            y_valid = y[train_n:]

            model.fit(X_train, y_train,
                      epochs=1,
                      validation_data=(X_valid, y_valid),
                      callbacks=callbacks_list,
                      verbose=100,
                     )

            y_ = model.predict(X_valid)
            res_gru.loc[n_steps, rnn_dim] += mean_squared_error(y_valid, y_)
res_gru /= len(ds.train_generator())

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [25]:
res_gru

,32,64,128,256
2,0.532979,0.503730,0.495337,0.480684
4,0.525646,0.489814,0.477247,0.463216
8,0.524070,0.484350,0.472073,0.460756
16,0.537259,0.490500,0.476918,0.465306
32,0.525607,0.482867,0.469963,0.458601


In [26]:
res_gru.mean(axis=0)

32     0.529112
64     0.490252
128    0.478307
256    0.465713
dtype: float64

In [28]:
res_gru.mean(axis=1)

2     0.503182
4     0.488981
8     0.485312
16    0.492496
32    0.484260
dtype: float64